In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
import os
import pandas as pd
corpus = []
vocabulary = []

entity_file = "NER_list.csv"
ent_file = pd.read_csv(entity_file)

vocabulary = []
for row in ent_file.iterrows():
    try:
        words = row[1]['entity'].split()
    except:
        words = [row[1]['entity']]
    for word in words:
        if word not in vocabulary:
            vocabulary.append(word)

True


In [7]:
ordered_list = []
temp_corp_list = []
window = 5

for root, dirs, files in os.walk("ExtractedGDELTData"):
   for dirname in sorted(dirs):
        ordered_list.append("ExtractedGDELTData/" + dirname + "/into_relex.txt")

for index in range(len(ordered_list[0:-(window - 1)])):
    temp_corp_list.append(ordered_list[index:index + window])

corpus = []
for corp in temp_corp_list:
    temp_corpus = ""
    for cor in corp:
        with open(cor, "r") as f:
            mylist = f.read().splitlines()
            for l in mylist:
                if l != "text" and l != "\n":
                    temp_corpus += " " + l
    corpus.append(temp_corpus)

pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
pipe['count'].transform(corpus).toarray()
tfidf = pipe.transform(corpus)

In [8]:
top_ners = 25
top_actants = np.argsort(tfidf.toarray(), axis=1)[:,-1*top_ners:]

with open("Results0423/tfidfedNER" + str(top_ners) + ".txt", "a+") as f:
    for row in top_actants:
        for r in row:
            f.write(vocabulary[r] + ", ")
        f.write("\n")